In [ ]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from datetime import datetime

# Connect to database and load all data
conn = sqlite3.connect('experiments.db')

# Load the full database
df = pd.read_sql_query("SELECT * FROM experiments", conn)
conn.close()

print(f"Total experiments: {len(df)}")
print(f"Database shape: {df.shape}")
print("\n" + "="*50)

# Display basic info
print("COLUMN INFO:")
print(df.info())
print("\n" + "="*50)

# Show first few rows (excluding the long text columns for readability)
display_cols = ['id', 'timestamp', 'd_model', 'num_heads', 'num_layers', 'd_ff', 
                'learning_rate', 'batch_size', 'val_loss', 'val_accuracy', 
                'val_path_validity', 'val_edge_accuracy', 'iteration']

print("FIRST 10 EXPERIMENTS:")
print(df[display_cols].head(10))
print("\n" + "="*50)

# Summary statistics
print("SUMMARY STATISTICS:")
numeric_cols = ['d_model', 'num_heads', 'num_layers', 'd_ff', 'learning_rate', 
                'val_loss', 'val_accuracy', 'val_path_validity', 'val_edge_accuracy', 'iteration']
print(df[numeric_cols].describe())
print("\n" + "="*50)

# Best and worst performers
print("TOP 5 BEST MODELS (lowest val_loss):")
best_models = df.nsmallest(5, 'val_loss')[display_cols]
print(best_models)
print("\n")

print("TOP 5 WORST MODELS (highest val_loss):")
worst_models = df.nlargest(5, 'val_loss')[display_cols]
print(worst_models)
print("\n" + "="*50)

# Iteration analysis
print("EXPERIMENTS PER ITERATION:")
iteration_counts = df.groupby('iteration').size()
print(iteration_counts)
print("\n")

print("BEST MODEL PER ITERATION:")
best_per_iteration = df.loc[df.groupby('iteration')['val_loss'].idxmin()]
print(best_per_iteration[['iteration', 'id', 'val_loss', 'd_model', 'num_heads', 'num_layers']])
print("\n" + "="*50)

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Distribution of validation loss
axes[0,0].hist(df['val_loss'], bins=20, alpha=0.7, edgecolor='black')
axes[0,0].set_title('Distribution of Validation Loss')
axes[0,0].set_xlabel('Validation Loss')
axes[0,0].set_ylabel('Frequency')

# 2. Val loss over time (by experiment ID)
axes[0,1].plot(df['id'], df['val_loss'], 'o-', alpha=0.6)
axes[0,1].set_title('Validation Loss Over Experiments')
axes[0,1].set_xlabel('Experiment ID')
axes[0,1].set_ylabel('Validation Loss')

# 3. Val loss by iteration
if 'iteration' in df.columns:
    df.boxplot(column='val_loss', by='iteration', ax=axes[1,0])
    axes[1,0].set_title('Validation Loss by Iteration')
    axes[1,0].set_xlabel('Iteration')

# 4. Correlation heatmap of key metrics
metrics = ['val_loss', 'val_accuracy', 'val_path_validity', 'val_edge_accuracy', 'd_model', 'num_heads', 'num_layers']
correlation_data = df[metrics].corr()
sns.heatmap(correlation_data, annot=True, cmap='coolwarm', center=0, ax=axes[1,1])
axes[1,1].set_title('Correlation Matrix')

plt.tight_layout()
plt.show()

# Show evolution progress if multiple iterations exist
if df['iteration'].nunique() > 1:
    print("EVOLUTION PROGRESS:")
    evolution_progress = df.groupby('iteration').agg({
        'val_loss': ['min', 'mean', 'std'],
        'val_accuracy': ['max', 'mean'],
        'val_path_validity': ['max', 'mean']
    }).round(4)
    print(evolution_progress)
    print("\n")
    
    # Plot evolution progress
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 3, 1)
    iteration_stats = df.groupby('iteration')['val_loss'].agg(['min', 'mean'])
    plt.plot(iteration_stats.index, iteration_stats['min'], 'o-', label='Best', color='green')
    plt.plot(iteration_stats.index, iteration_stats['mean'], 'o-', label='Mean', color='blue')
    plt.title('Val Loss Evolution')
    plt.xlabel('Iteration')
    plt.ylabel('Validation Loss')
    plt.legend()
    
    plt.subplot(1, 3, 2) 
    iteration_stats = df.groupby('iteration')['val_accuracy'].agg(['max', 'mean'])
    plt.plot(iteration_stats.index, iteration_stats['max'], 'o-', label='Best', color='green')
    plt.plot(iteration_stats.index, iteration_stats['mean'], 'o-', label='Mean', color='blue')
    plt.title('Val Accuracy Evolution')
    plt.xlabel('Iteration')
    plt.ylabel('Validation Accuracy')
    plt.legend()
    
    plt.subplot(1, 3, 3)
    iteration_stats = df.groupby('iteration')['val_path_validity'].agg(['max', 'mean'])
    plt.plot(iteration_stats.index, iteration_stats['max'], 'o-', label='Best', color='green')
    plt.plot(iteration_stats.index, iteration_stats['mean'], 'o-', label='Mean', color='blue')
    plt.title('Path Validity Evolution')
    plt.xlabel('Iteration')
    plt.ylabel('Path Validity')
    plt.legend()
    
    plt.tight_layout()
    plt.show()

# Hyperparameter analysis
print("HYPERPARAMETER ANALYSIS:")
print("\nBest d_model values:")
print(df.nsmallest(10, 'val_loss')['d_model'].value_counts())

print("\nBest num_heads values:")
print(df.nsmallest(10, 'val_loss')['num_heads'].value_counts())

print("\nBest learning_rate values:")
print(df.nsmallest(10, 'val_loss')['learning_rate'].value_counts())

# Display the complete dataframe at the end for detailed inspection
print("\n" + "="*50)
print("COMPLETE DATABASE (use df variable for further analysis):")
print(f"Access with: df.head(), df.tail(), df[df['val_loss'] < 1.0], etc.")
print(f"Available columns: {list(df.columns)}")

# Make df available for interactive use
globals()['df'] = df